# Nonlinear Forecasting

In [1]:
import importlib

# List of libraries to check
libraries = [
    'numpy',
    'matplotlib',
    'scipy',
    'torch',
    'torchdiffeq',
    'torchsummary',
    'minepy',
    'pyrqa',
    'pyts',
    'MFDFA',
    'pyinform',
    'graphviz',
    'fa2',
    'networkx'
]

for lib in libraries:
    try:
        module = importlib.import_module(lib)
        version = getattr(module, '__version__', 'Unknown version')
        print(f'{lib}: {version}')
    except ImportError:
        print(f'{lib} is not installed.')

numpy: 1.24.3
matplotlib: 3.7.0
scipy: 1.10.1
torch: 2.0.1+cu117
torchdiffeq: 0.2.3
torchsummary: Unknown version
minepy: b'1.2.6'
pyrqa: Unknown version
pyts: 0.12.0
MFDFA: 0.4.3
pyinform: Unknown version
graphviz: 0.20.1
fa2 is not installed.
networkx: 3.0


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert

# Load EEG data
eeg_data = np.loadtxt("eeg_data.txt")

# Define TES stimulation parameters
stimulation_params = {
    "frequency": 10,  # Hz
    "amplitude": 1.5,  # mA
    "duration": 5  # minutes
}

# Calculate phase synchronization for the EEG data and TES stimulation
sampling_rate = 1000  # Hz
eeg_phase = np.angle(hilbert(eeg_data))

stimulation_frequency = stimulation_params["frequency"]
stimulation_duration = stimulation_params["duration"] * 60  # Convert to seconds
stimulation_waveform = stimulation_params["amplitude"] * np.sin(2 * np.pi * stimulation_frequency * np.arange(stimulation_duration))

stimulation_phase = np.angle(hilbert(stimulation_waveform))

# Compute the phase difference between EEG data and TES stimulation
phase_diff = np.angle(np.exp(1j * (stimulation_phase - eeg_phase)))

# Calculate the phase synchronization
sync_strength = np.abs(np.mean(np.exp(1j * phase_diff)))

# Visualize the results
plt.subplot(2, 2, 1)
plt.plot(eeg_data)
plt.title("EEG data")

plt.subplot(2, 2, 2)
plt.plot(stimulation_waveform)
plt.title("TES stimulation")

plt.subplot(2, 2, 3)
plt.plot(phase_diff)
plt.title("Phase difference")

plt.subplot(2, 2, 4)
plt.bar([0, 1], [1 - sync_strength, sync_strength])
plt.xticks([0, 1], ["Desynchronized", "Synchronized"])
plt.title("Phase synchronization")

plt.show()

print("Phase synchronization strength:", sync_strength)

OSError: eeg_data.txt not found.

In [ ]:
import numpy as np
from nolds import lyap_e, pfd
from pyts.approximation import SymbolicAggregateApproximation
from pyts.datasets import load_gunpoint
from pyts.multivariate.image import JointRecurrencePlot
from pyts.multivariate.transformation import MultivariateTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor


# Load EEG data
eeg_data = np.loadtxt("eeg_data.txt")

# Define TES stimulation parameters
stimulation_params = {
    "frequency": 10,  # Hz
    "amplitude": 1.5,  # mA
    "duration": 5  # minutes
}

# Apply TES stimulation
stimulation_duration = stimulation_params["duration"] * 60  # Convert to seconds
stimulation_frequency = stimulation_params["frequency"]
stimulation_amplitude = stimulation_params["amplitude"]
stimulation_waveform = stimulation_amplitude * np.sin(2 * np.pi * stimulation_frequency * np.arange(stimulation_duration))

# Add TES stimulation to the EEG data
eeg_data_with_stimulation = eeg_data + stimulation_waveform

# Calculate the fractal dimension of the EEG data with TES stimulation
fd_with_stimulation = pfd(eeg_data_with_stimulation)

# Apply nonlinear forecasting using Takens' embedding theorem
tau = 10
embedding_dimension = 3
X = np.array([eeg_data_with_stimulation[i:i + embedding_dimension] for i in range(0, len(eeg_data_with_stimulation) - embedding_dimension - tau, tau)])
y = eeg_data_with_stimulation[embedding_dimension + tau:len(eeg_data_with_stimulation):tau]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use symbolic aggregate approximation to transform the data
n_bins = 4
sax = SymbolicAggregateApproximation(n_bins=n_bins, strategy='uniform')
X_train_sax = sax.fit_transform(X_train)
X_test_sax = sax.transform(X_test)

# Use joint recurrence plot to transform the data
n_neighbors = 2
threshold = 0.1
weights = 'distance'
jrp = JointRecurrencePlot(n_neighbors=n_neighbors, threshold=threshold, weights=weights)
X_train_jrp = jrp.fit_transform(X_train_sax)
X_test_jrp = jrp.transform(X_test_sax)

# Use multivariate transformation to transform the data
mt = MultivariateTransformer(norm_mean=True, norm_std=True)
X_train_mt = mt.fit_transform(X_train_jrp)
X_test_mt = mt.transform(X_test_jrp)

# Train a K-nearest neighbors regression model
n_neighbors = 10
weights = 'distance'
knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights)
knn.fit(X_train_mt, y_train)

# Make predictions using the trained model
y_pred = knn.predict(X_test_mt)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)